In [1]:
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

Using Theano backend.


In [2]:
import keras.backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [3]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [6]:
# Function to create model, required for KerasClassifier
model = Sequential()
model.add(Dense(12, input_dim=9, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_score,recall,precision])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision, recall])

In [7]:
seed=42

In [8]:
np.random.seed(seed)

In [9]:
import pandas as pd
df = pd.read_csv("wbc.csv")

In [10]:
df=df.replace('?',np.nan)
df['bare_nuclei'] = pd.to_numeric(df.bare_nuclei.fillna(df.bare_nuclei.median()))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
clump_thickness       699 non-null int64
cell_size             699 non-null int64
cell_shape            699 non-null int64
marginal_adhesion     699 non-null int64
epitelial_cellsize    699 non-null int64
bare_nuclei           699 non-null float64
bland_chromatin       699 non-null int64
normal_nucleoli       699 non-null int64
mitoses               699 non-null int64
jenis                 699 non-null int64
dtypes: float64(1), int64(9)
memory usage: 54.7 KB


In [11]:
X = df.drop([df.columns[-1]], axis=1)
y = df.jenis.map({2: 0, 4: 1})

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

In [13]:
y_train = np_utils.to_categorical(y_train)
y_train.shape

(468L, 2L)

In [14]:
y_test = np_utils.to_categorical(y_test)
y_test.shape

(231L, 2L)

In [15]:
y_test

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,

In [16]:
model.fit(X_train.as_matrix(), y_train)

Epoch 1/10
468/468 [==============================] - 0s - loss: 1.0578 - acc: 0.3632 - f1_score: 0.3632 - precision: 0.3632 - recall: 0.3632     
Epoch 2/10
468/468 [==============================] - 0s - loss: 0.9240 - acc: 0.3675 - f1_score: 0.3675 - precision: 0.3675 - recall: 0.3675     
Epoch 3/10
468/468 [==============================] - 0s - loss: 0.8314 - acc: 0.4487 - f1_score: 0.4487 - precision: 0.4487 - recall: 0.4487     
Epoch 4/10
468/468 [==============================] - 0s - loss: 0.7635 - acc: 0.4893 - f1_score: 0.4893 - precision: 0.4893 - recall: 0.4893     

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn import metrics

In [18]:
scores = model.evaluate(X_test.as_matrix(), y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/231 [===>..........................] - ETA: 0s
acc: 84.85%


In [17]:
predict = model.predict(X_test.as_matrix())

In [18]:
ypreds = np.argmax(predict, axis=1)
print(average_precision_score(ytrue, ypreds))
print(accuracy_score(ytrue, ypreds))

NameError: name 'average_precision_score' is not defined

In [19]:
metrics.confusion_matrix(y_test2, predict)

ValueError: Can't handle mix of multilabel-indicator and continuous-multioutput

In [18]:
from sklearn.metrics import classification_report

In [20]:
report = classification_report(y_test, predict)
print(report)

ValueError: Mix type of y not allowed, got types set(['continuous-multioutput', 'multilabel-indicator'])

In [17]:
accuracy = accuracy_score(y_test, predict)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred3)
print (('Akurasi: %.4f') % (accuracy))
print (('Precision: %.4f') % (precision[1]))
print (('Recall: %.4f') % (recall[1]))
print (('F1: %.4f') % (f1[1]))

ValueError: Can't handle mix of multilabel-indicator and continuous-multioutput